Implémentation de résolution numérique approchée de l'équation 1 :
$\frac{dy}{dx} = f(x,y), 
y(a) = A$
à l'aide de la la méthode de descente de gradients sur la fonction d'erreur


In [13]:
#imports
from numpy import pi, cos, sin, dot, zeros, linspace, arange
from numpy.random import randn

Paramètres du modèle

In [14]:
#nombre de coefficients de Fourier ajustables
M = 10
M_range = arange(M) + 1

# nombre de points pour la variable indépendante
N = 100
X = linspace(0,1,N)

Définition du gradient

In [15]:
def calcGrad(A) :
    #Calcule le gradient de l'erreur par rapport au vecteur
    #des coefficients
    grad = zeros((M))
    V = cos(2*pi*X)
    for m in range(1, M+1) :
        for i in range(N) :
            V[i] += 2*pi*m*A[m-1]*cos(2*pi*m*X[i])
    for l in range(1, M+1) :
        W = pi*l*cos(2*pi*l*X)
        grad[l-1] = dot(V,W)
    return grad

Paramètres d'apprentissage

In [16]:
alpha = 1e-5 #taux d'apprentissage pour la descente de gradients
epochs = 1000 #nombre d'itération

#initialisation des coefficients
A = randn((M))

Entraînement

In [17]:
for k in range(epochs) :
    if k%100==0 :
        print("Coefficients après",k,"itérations :",A)
    A -= alpha*calcGrad(A)

Coefficients après 0 itérations : [ 0.27395343 -0.91915053  0.34538113 -1.22352381 -0.20884775  0.52616903
  1.80334578  2.25662669 -1.19705469  1.40139722]
Coefficients après 100 itérations : [ 0.00149605 -0.01790234 -0.00066131 -0.00054355 -0.00043674 -0.00036395
 -0.00031175 -0.0002726  -0.00024218 -0.00021786]
Coefficients après 200 itérations : [-0.099908   -0.0009499  -0.00037056 -0.00026417 -0.00020651 -0.00016997
 -0.00014461 -0.00012593 -0.00011157 -0.00010018]
Coefficients après 300 itérations : [-1.37327022e-01 -2.52161442e-04 -1.38536628e-04 -9.84335671e-05
 -7.68724622e-05 -6.32424584e-05 -5.37936299e-05 -4.68371374e-05
 -4.14926177e-05 -3.72534948e-05]
Coefficients après 400 itérations : [-1.51113421e-01 -9.12036719e-05 -5.10724837e-05 -3.62826392e-05
 -2.83338916e-05 -2.33096069e-05 -1.98267703e-05 -1.72626849e-05
 -1.52927915e-05 -1.37303469e-05]
Coefficients après 500 itérations : [-1.56192410e-01 -3.35705023e-05 -1.88159362e-05 -1.33670197e-05
 -1.04385697e-05 -8.5875

Affichage des résultats

In [18]:
print()
print("Coefficients finaux après",epochs,"itérations :",A)


Coefficients finaux après 1000 itérations : [-1.59134839e-01 -2.27805113e-07 -1.27688232e-07 -9.07108999e-08
 -7.08379259e-08 -5.82765863e-08 -4.95690823e-08 -4.31585747e-08
 -3.82336195e-08 -3.43273348e-08]
